In [6]:
import ee 
import geemap

ee.Initialize(project="ee-india-reservoirs")

import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

from utils import wapor_ETo
from utils.wapor_ETo import load_wapor_ETo

aoi_selected=ee.FeatureCollection('projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box').geometry()

first_year = 2018
last_year = 2023
WAPOR_ETo_decadal = load_wapor_ETo(
    first_year, last_year, frequency="dekadal"
)#

# Load the FeatureCollection
Kcs = ee.FeatureCollection('projects/thurgau-irrigation/assets/FribourgAndVaud/ETc_WAPOR/Kc_Pasture_Broye_Aquastat')

# Print FeatureCollection and properties
#print('Kcs:', Kcs.getInfo())

# Define the WAPOR_ETc_decadal calculation
def compute_ETc(img):
    img = ee.Image(img)
    m = ee.Number(img.get('Month'))
    decade = ee.Number(img.get('dekade'))

    # Get the Kc value based on Month and Decade
    Kc_feature = Kcs.filter(
        ee.Filter.And(ee.Filter.eq('Month', m), ee.Filter.eq('Decade', decade))
    ).first()
    Kc = ee.Number(Kc_feature.get('Kc'))

    # Compute ETc
    return img.multiply(Kc).float().rename('ETc').copyProperties(img).set(
        'system:time_start', img.get('system:time_start')
    )

WAPOR_ETc_decadal = WAPOR_ETo_decadal.map(compute_ETc)

etc_collection = (
    WAPOR_ETc_decadal
    .filterBounds(aoi_selected.centroid())
    #.filterDate(f"{YEAR}-04-01", f"{YEAR}-10-31")
    .sort("system:time_start")
)

# Define the function to extract ETc values and convert images to features
def convert_to_feature(img):
    # Extract the ETc value at the centroid of the AOI using reduceRegion
    ETc_value = img.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=aoi_selected.centroid()
    ).values().get(0)  # Get the first value from the dictionary

    # Create a feature with the geometry, properties, and the ETc value
    return ee.Feature(aoi_selected.centroid()).copyProperties(img).set(
        'ETc', ETc_value
    ).set(
        'system:time_start', img.get('system:time_start')
    )

# Apply the function to the ImageCollection and convert it to a FeatureCollection
WAPOR_ETc_decadal_values = ee.FeatureCollection(
    etc_collection.map(convert_to_feature)
)

# Print the resulting FeatureCollection
print(WAPOR_ETc_decadal_values.filter(ee.Filter.eq('Year',2022)).filter(ee.Filter.eq('Month',8)).first().getInfo())

# Define the export function
def export_feature_collection_to_asset(fc, asset_id, description):
    """
    Export a FeatureCollection to an Earth Engine Asset.

    Args:
        fc (ee.FeatureCollection): The FeatureCollection to export.
        asset_id (str): The destination asset ID.
        description (str): A human-readable name for the export task.
    """
    task = ee.batch.Export.table.toAsset(
        collection=fc,
        description=description,
        assetId=asset_id
    )
    task.start()
    print(f"Export started: {description}, to asset: {asset_id}")


# Export the FeatureCollection to an Earth Engine asset
export_feature_collection_to_asset(
    fc=WAPOR_ETc_decadal_values,
    asset_id="projects/thurgau-irrigation/assets/FribourgAndVaud/ETc_WAPOR/ETc_Pasture_Broye",
    description="ETc_Pasture_Broye_Export"
)



{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [6.8748218261196365, 46.78699674759434]}, 'id': '165', 'properties': {'ETc': 50.51612854003906, 'Month': 8, 'Year': 2022, 'dekade': 1, 'system:time_start': 1659744000000}}
Export started: ETc_Pasture_Broye_Export, to asset: projects/thurgau-irrigation/assets/FribourgAndVaud/ETc_WAPOR/ETc_Pasture_Broye
